In [1]:
import sys

def config_paths(user, env_name):
    paths = ['',
             '/home/{0}/{1}/.env/bin'.format(user, env_name),
             '/usr/lib/python35.zip',
             '/usr/lib/python3.5',
             '/usr/lib/python3.5/plat-x86_64-linux-gnu',
             '/usr/lib/python3.5/lib-dynload',
             '/home/{0}/{1}/.env/lib/python3.5/site-packages'.format(user, env_name),
             '/home/{0}/{1}/.env/lib/python3.5/site-packages/IPython/extensions'.format(user, env_name),
             '/home/{0}/.ipython']

    for path in paths:
        sys.path.append(path)
        
config_paths('omri', 'my_gpu')

import os
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt
import pandas as pd

from keras.models import Sequential
from keras import Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

TRAIN_DIR = '../'
K = 30
SAMPLE_NUM = 128 * K   
IMG_SIZE = 197

from keras.models import model_from_json




/home/omri/my_gpu/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
load_model = True

In [3]:
if load_model:
    json_file = open('my_vgg_all_1_epoch.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)

    # load weights into new model
    model.load_weights("my_vgg_all_1_epoch.h5")

else:
    from keras.applications.vgg16 import VGG16
    from keras.applications.resnet50 import ResNet50
    from keras import regularizers

    model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(IMG_SIZE, IMG_SIZE, 3),
                     pooling=None, classes=128)

    # model = Model(model.input, preds)

    for layer in model.layers:
        layer.trainable = False

    last = model.output

    x = Flatten()(last)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='sigmoid',kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.001))(x)
    preds = Dense(128, activation='softmax')(x)

    model = Model(model.input, preds)

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 197, 197, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 197, 197, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 197, 197, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 98, 98, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 98, 98, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 98, 98, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 49, 49, 128)       0         
__________

# Train on all data

In [ ]:
def train_on_chunk(model, chunk_number, X_validation='', Y_validation=''):
    global K
    global TRAIN_DIR
    global SAMPLE_NUM
    global IMG_SIZE
    X_train = np.zeros(shape = (SAMPLE_NUM, IMG_SIZE,IMG_SIZE,3))
    Y_train = np.zeros(shape = (SAMPLE_NUM,))

    for category_ind in range(1,129):
        cagetory_dir = os.path.join(TRAIN_DIR, str(category_ind))

        cur_image_list = os.listdir(cagetory_dir)
        if (chunk_number+1)*K > len(cur_image_list):
            temp_list = cur_image_list[chunk_number*K:]
        else:
            temp_list = cur_image_list[chunk_number*K:(chunk_number+1)*K]
        for im_ind, im_name in enumerate(temp_list):
            im = cv2.imread(os.path.join(cagetory_dir, im_name))
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            im = cv2.resize(im, (IMG_SIZE, IMG_SIZE))
            im = im/255.0

            X_train[(category_ind-1)*K + im_ind] = im
            Y_train[(category_ind-1)*K + im_ind] = category_ind - 1

    random_indices = np.random.permutation(SAMPLE_NUM)
#     X_total = X_total[random_indices]
#     Y_total = Y_total[random_indices]
    
    # Get Validation from first iteration
    if chunk_number == 0:
        return X_train, np_utils.to_categorical(Y_train,128)
    
    # If not first iteration, continue training
    else:
        train_num = int(SAMPLE_NUM * 1)
        Y_train = np_utils.to_categorical(Y_train,128)
        X_val = X_validation
        Y_val = Y_validation

    
    earlystopper = EarlyStopping(patience=5, verbose=1)
    checkpointer = ModelCheckpoint('model_iter_dropout.h5', verbose=1, save_best_only=True)
    
    epochs = 1
#     learning_rate = 0.001
#     decay_rate = learning_rate / epochs

    sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    # adam = optimizers.Adam(lr=0.01)

    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

    # model.fit_generator(image_gen.flow(X_train, Y_train, batch_size=32),
    #           epochs=40,
    #           verbose=1,
    #           class_weight=class_weight,
    #             validation_data=(X_val, Y_val))

    
    model.fit(X_train, Y_train, batch_size=32, 
              epochs=epochs,verbose=1, validation_data=(X_val, Y_val), callbacks=[earlystopper])
   

    
for i in range(100):    
    for chunk in range(30) :
        print(i, chunk)
        if (chunk == 0) & (i == 0):
            X_val, Y_val = train_on_chunk(model, chunk)
        else:
            train_on_chunk(model, chunk, X_val, Y_val)
        
        model_json = model.to_json()
        with open("my_vgg_all_1_epoch_2.json", "w") as json_file:
            json_file.write(model_json)
        model.save_weights("my_vgg_all_2_epoch_2.h5")

0 0


In [ ]:
model_json = model.to_json()
with open("my_vgg_all_1_epoch.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("my_vgg_all_1_epoch.h5")